In [1]:
import numpy as np
import pandas as pd

In [70]:
# Store all numbers of stores in a list
store_restaurant = pd.read_excel('store_restaurant.xlsx')
store_number  = store_restaurant.STORE_NUMBER
print(store_number)

0    46673
1     4904
2    12631
3    20974
Name: STORE_NUMBER, dtype: int64


## Match the order, keep the Recipe Id

In [71]:
menuitem = pd.read_csv('menuitem.csv') 
menu_items = pd.read_csv('menu_items.csv') 
menu_items = menu_items.rename(columns={'MenuItemId': 'Id'})
orders = pd.merge(menuitem, menu_items, how="left", on=["PLU", "Id"]).drop(columns=['PLU','Id'])
orders

,StoreNumber,date,RecipeId
0,46673,2015/3/5,358
1,46673,2015/3/5,305
2,12631,2015/3/5,47
3,12631,2015/3/5,22
4,12631,2015/3/5,579
...,...,...,...
91426,4904,2015/6/15,302
91427,4904,2015/6/15,109
91428,4904,2015/6/15,107
91429,12631,2015/6/15,40


## Match the main ingredient and the quantity

In [91]:
recipe_ingredient_assignments = pd.read_csv('recipe_ingredient_assignments.csv')
orders_with_ingredient = pd.merge(orders, recipe_ingredient_assignments, how="left", on=["RecipeId"])
orders_with_ingredient

,StoreNumber,date,RecipeId,IngredientId,Quantity
0,46673,2015/3/5,358,3.0,5.0
1,46673,2015/3/5,305,97.0,1.0
2,46673,2015/3/5,305,100.0,1.0
3,46673,2015/3/5,305,110.0,2.0
4,46673,2015/3/5,305,147.0,1.0
...,...,...,...,...,...
177023,12631,2015/6/15,40,9.0,2.0
177024,46673,2015/6/15,305,97.0,1.0
177025,46673,2015/6/15,305,100.0,1.0
177026,46673,2015/6/15,305,110.0,2.0


## Match the sub ingredient and the quantity

In [101]:
recipe_sub_recipe_assignments = pd.read_csv('recipe_sub_recipe_assignments.csv')
orders_with_sub_ingredient = pd.merge(orders, recipe_sub_recipe_assignments, how="left", on=["RecipeId"])
orders_with_sub_ingredient = orders_with_sub_ingredient.dropna()
orders_with_sub_ingredient

,StoreNumber,date,RecipeId,SubRecipeId,Factor
0,46673,2015/3/5,358,1.0,1.0
1,46673,2015/3/5,358,9.0,1.0
2,46673,2015/3/5,358,10.0,1.0
3,46673,2015/3/5,358,12.0,1.0
5,12631,2015/3/5,47,9.0,1.0
...,...,...,...,...,...
211232,4904,2015/6/15,107,12.0,1.0
211233,12631,2015/6/15,40,1.0,1.0
211234,12631,2015/6/15,40,9.0,1.0
211235,12631,2015/6/15,40,10.0,1.0


In [84]:
sub_recipe_ingr_assignments = pd.read_csv('sub_recipe_ingr_assignments.csv')
orders_with_sub_ingredient_1 = pd.merge(orders_with_sub_ingredient, sub_recipe_ingr_assignments, how="left", on=["SubRecipeId"])
orders_with_sub_ingredient_1

,StoreNumber,date,RecipeId,SubRecipeId,Factor,IngredientId,Quantity
0,46673,2015/3/5,358,1.0,1.0,1,1
1,46673,2015/3/5,358,1.0,1.0,135,2
2,46673,2015/3/5,358,1.0,1.0,138,1
3,46673,2015/3/5,358,1.0,1.0,146,1
4,46673,2015/3/5,358,1.0,1.0,157,1
...,...,...,...,...,...,...,...
924549,12631,2015/6/15,40,12.0,1.0,37,0
924550,12631,2015/6/15,40,12.0,1.0,39,0
924551,12631,2015/6/15,40,12.0,1.0,43,0
924552,12631,2015/6/15,40,12.0,1.0,122,0


In [85]:
orders_with_sub_ingredient_2 = orders_with_sub_ingredient_1.copy()
orders_with_sub_ingredient_2['Quantity(factor)'] = orders_with_sub_ingredient_2.Factor *  orders_with_sub_ingredient_2.Quantity
orders_with_sub_ingredient_2

,StoreNumber,date,RecipeId,SubRecipeId,Factor,IngredientId,Quantity,Quantity(factor)
0,46673,2015/3/5,358,1.0,1.0,1,1,1.0
1,46673,2015/3/5,358,1.0,1.0,135,2,2.0
2,46673,2015/3/5,358,1.0,1.0,138,1,1.0
3,46673,2015/3/5,358,1.0,1.0,146,1,1.0
4,46673,2015/3/5,358,1.0,1.0,157,1,1.0
...,...,...,...,...,...,...,...,...
924549,12631,2015/6/15,40,12.0,1.0,37,0,0.0
924550,12631,2015/6/15,40,12.0,1.0,39,0,0.0
924551,12631,2015/6/15,40,12.0,1.0,43,0,0.0
924552,12631,2015/6/15,40,12.0,1.0,122,0,0.0


In [88]:
orders_with_sub_ingredient_3 = orders_with_sub_ingredient_2.copy()
orders_with_sub_ingredient_3=orders_with_sub_ingredient_3.drop(columns=['Factor','Quantity','SubRecipeId'])
orders_with_sub_ingredient_3 = orders_with_sub_ingredient_3.rename(columns={'Quantity(factor)':'Quantity'})
orders_with_sub_ingredient_3

,StoreNumber,date,RecipeId,IngredientId,Quantity
0,46673,2015/3/5,358,1,1.0
1,46673,2015/3/5,358,135,2.0
2,46673,2015/3/5,358,138,1.0
3,46673,2015/3/5,358,146,1.0
4,46673,2015/3/5,358,157,1.0
...,...,...,...,...,...
924549,12631,2015/6/15,40,37,0.0
924550,12631,2015/6/15,40,39,0.0
924551,12631,2015/6/15,40,43,0.0
924552,12631,2015/6/15,40,122,0.0


## Append the main ingredient and sub ingredient

In [103]:
orders_with_all_ingredient = pd.concat([orders_with_sub_ingredient_3,orders_with_ingredient])
orders_with_all_ingredient = orders_with_all_ingredient[orders_with_all_ingredient.Quantity != 0]
orders_with_all_ingredient = orders_with_all_ingredient.drop(columns=['RecipeId'])
orders_with_all_ingredient

,StoreNumber,date,IngredientId,Quantity
0,46673,2015/3/5,1.0,1.0
1,46673,2015/3/5,135.0,2.0
2,46673,2015/3/5,138.0,1.0
3,46673,2015/3/5,146.0,1.0
4,46673,2015/3/5,157.0,1.0
...,...,...,...,...
177023,12631,2015/6/15,9.0,2.0
177024,46673,2015/6/15,97.0,1.0
177025,46673,2015/6/15,100.0,1.0
177026,46673,2015/6/15,110.0,2.0


## Calculate the demand of lettuce

In [114]:
orders_with_all_lettuce = orders_with_all_ingredient[orders_with_all_ingredient.IngredientId == 27]

In [120]:
lettuce_demand = pd.DataFrame(data=orders_with_all_lettuce.groupby(by=['StoreNumber','date'])['Quantity'].sum())
lettuce_demand = lettuce_demand.reset_index()
lettuce_demand

,StoreNumber,date,Quantity
0,4904,2015/3/13,148.0
1,4904,2015/3/14,180.0
2,4904,2015/3/15,345.0
3,4904,2015/3/16,394.0
4,4904,2015/3/17,396.0
...,...,...,...
389,46673,2015/6/5,94.0
390,46673,2015/6/6,78.0
391,46673,2015/6/7,242.0
392,46673,2015/6/8,164.0


In [124]:
lettuce_demand.to_csv("lettuce_demand.csv")